In this tutorial I'll be demonstrating how to fine tune a pretrained model to perform complex tasks such as reasoning and completion etc

In [1]:
#Getting started with lamini
import lamini
import jsonlines
import itertools
import pandas as pd
from pprint import pprint
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Trainer
import datasets
import torch
from datasets import load_dataset
import logging
from transformers import TrainingArguments
logger = logging.getLogger(__name__)

d:\Dhruv_Sandhu\Machine Learning\Projects\Fine_tuning\finenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

lamini.api_key = "9830b83cdb2c1ceecf64b18ca4172743376e23bc275ed4112dbdf01658b9ad6d"

In [3]:
llm_nf = lamini.Lamini("meta-llama/Llama-2-7b-hf")

In [4]:
#Example demonstrating very poor performance
print(llm_nf.generate("Tell me how to train my dog to sit"))
print("XXXXXXXXXX")
print(llm_nf.generate("What do you think of mars"))
print("XXXXXXXXXX")
print(llm_nf.generate("""Agent: I'm here to help you with your Amazon deliver order.
Customer: I didn't get my item
Agent: I'm sorry to hear that. Which item was it?
Customer: the blanket
Agent:"""))

.
Tell me how to train my dog to sit. I have a 10 month old puppy and I want to train him to sit. I have tried the treat method and he just sits there and looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" c

In [5]:
#Lets compare to a finetuned model that has been finetuned by some other person
llm_f=lamini.Lamini("meta-llama/Llama-2-7b-chat-hf")

In [6]:
#Far better result as than non finetuned model
print(llm_f.generate("[INST]Tell me how to train my dog to sit[/INST]"))
print("XXXXXXXXXX")
print(llm_f.generate("What do you think of the planet mars"))
print("XXXXXXXXXX")
print(llm_f.generate("""Agent: I'm here to help you with your Amazon deliver order.
Customer: I didn't get my item
Agent: I'm sorry to hear that. Which item was it?
Customer: the blanket
Agent:"""))

 Training your dog to sit is a basic obedience command that can be achieved with patience, consistency, and positive reinforcement. Here's a step-by-step guide on how to train your dog to sit:

1. Choose a quiet and distraction-free area: Find a quiet area with no distractions where your dog can focus on you.
2. Have treats ready: Choose your dog's favorite treats and have them ready to use as rewards.
3. Stand in front of your dog: Stand in front of your dog and hold a treat close to their nose.
4. Move the treat up and back: Slowly move the treat up and back, towards your dog's tail, while saying "sit" in a calm and clear voice.
5. Dog will sit: As you move the treat, your dog will naturally sit down to follow the treat. The moment their bottom touches the ground, say "good sit" and give them the treat.
6. Repeat the process: Repeat steps 3-5 several times, so your dog starts to associate the command "sit" with the action of sitting down.
7. Gradually phase out the treats: As your do

In [7]:
#Lets explore the dataset that we want to finetune our model on
#Here we are using the dataset alpaca from tatsu-lab
#Alpaca is a dataset of 52,000 instructions and demonstrations generated by OpenAI's text-davinci-003 engine. 
#This instruction data can be used to conduct instruction-tuning for language models and make the language model follow instruction better.
pretrained_dataset = load_dataset("tatsu-lab/alpaca", split="train", streaming=True)

In [8]:
print(pretrained_dataset.column_names)

['instruction', 'input', 'output', 'text']


Here text is the default prompt template provided the makers of the dataset but we will create our own prompt since we like to work more :)

In [9]:
count=0
for _ in pretrained_dataset:
    count+=1
print(count)

52002


In [10]:
n = 3
print("Pretrained dataset:")
top_n = list(itertools.islice(pretrained_dataset, n))
for i in top_n:
  print(i)
  print()
#Its output basically consists of instructions and response

Pretrained dataset:
{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}

{'instruction': 'What are the three primary colors?', 'input': '', 'output': 'The three primary colors are red, blue, and yellow.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat are the three primary

Below is the different types of prompt template that we can use

But we will create prompt according to our use case

In [11]:
prompt_template_with_input = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""

In [12]:
prompt_template_without_input = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [13]:
processed_data = []
for j in top_n:
  if j["input"]:#if the input is there in the datapoint that use prompt template that handles the input
    processed_prompt = prompt_template_with_input.format(instruction=j["instruction"],input=j["input"])
  else:#If the input is not there in the datapoint then use the other prompt template that works on just instructions
    processed_prompt = prompt_template_without_input.format(instruction=j["instruction"])

  processed_data.append({"input": processed_prompt, "output": j["output"]})


In [14]:
#It tells what the model is gonna get that is input and what is it required to be outputted from the model which is given as output
#This is what goes into the model
pprint(processed_data[1])

{'input': 'Below is an instruction that describes a task. Write a response '
          'that appropriately completes the request.\n'
          '\n'
          '### Instruction:\n'
          'What are the three primary colors?\n'
          '\n'
          '### Response:',
 'output': 'The three primary colors are red, blue, and yellow.'}


Save data to Jsonl

In [15]:
with jsonlines.open(f'alpaca_processed.jsonl', 'w') as writer:
    writer.write_all(processed_data)

In [16]:
#Both the dataset it lamini/alpaca and tatsu-labs/alpaca are same and contain 52002 datapoints in it.
#It just the structured form of it 
dataset_path_hf = "lamini/alpaca"
dataset_hf = load_dataset(dataset_path_hf)
pprint(dataset_hf)

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 52002
    })
})


Example Demonstrating the how to use an example at inference time.

In [17]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")

d:\Dhruv_Sandhu\Machine Learning\Projects\Fine_tuning\finenv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
#These are some heavy models and take huge time.
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

Below is an inference function which shows the working of how a particular datapoint is given to a finetuned as well as non-finetuned model
This is not for training it just how the model deals with an input at the inference time

In [19]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",#give the output in pytorch tensors
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [20]:
finetuning_dataset_path = "lamini/lamini_docs"
finetuning_dataset = load_dataset(finetuning_dataset_path)
print(finetuning_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
})


In [21]:
test_sample = finetuning_dataset["test"][0]
print(test_sample)

pprint(inference(test_sample["question"], model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{'question': 'Can Lamini generate technical documentation or user manuals for software projects?', 'answer': 'Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.', 'input_ids': [5804, 418, 4988, 74, 6635, 7681, 10097, 390, 2608, 11595, 84, 323, 3694, 6493, 32, 4374, 13, 418, 4988, 74, 476, 6635, 7681, 10097, 285, 2608, 11595, 84, 323, 3694, 6493, 15, 733, 4648, 3626, 3448, 5978, 5609, 281, 2794, 2590, 285, 44003, 10097, 326, 310, 3477, 281, 2096, 323, 1097, 7681, 285, 1327, 14, 48746, 4212, 15, 831, 476, 5321, 12259, 247, 1534, 2408, 273, 673, 285, 3434, 275, 6153, 10097, 13, 6941, 731, 281, 2770, 327, 643, 7794, 273, 616, 6493, 15], 'attention

In [22]:
pprint(test_sample["question"])

('Can Lamini generate technical documentation or user manuals for software '
 'projects?')


In [23]:
instruction_model = AutoModelForCausalLM.from_pretrained("lamini/lamini_docs_finetuned")

In [24]:
pprint(inference(test_sample["question"], instruction_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


('Yes, Lamini can generate technical documentation or user manuals for '
 'software projects. This can be achieved by providing a prompt for a specific '
 'technical question or question to the LLM Engine, or by providing a prompt '
 'for a specific technical question or question. Additionally, Lamini can be '
 'trained on specific technical questions or questions to help users '
 'understand the process and provide feedback to the LLM Engine. Additionally, '
 'Lamini')


In [25]:
model=lamini.Lamini("EleutherAI/pythia-70m")

In [26]:
pprint(model.generate("How to train my dog?"))

('\n'
 '\n'
 "I'm not sure if I'm going to be able to train my dog.\n"
 '\n'
 'A:\n'
 '\n'
 "I'm not sure if I'm going to be able to train my dog.\n"
 '\n'
 'A:\n'
 '\n'
 "I'm not sure if I'm going to be able to train my dog.\n"
 '\n'
 'A:\n'
 '\n'
 "I'm not sure if I'm going to be able to train my dog.\n"
 '\n'
 'A:\n'
 '\n'
 "I'm not sure if I'm going to be able to train my dog.\n"
 '\n'
 'A:\n'
 '\n'
 "I'm not sure if I'm going to be able to train my dog.\n"
 '\n'
 'A:\n'
 '\n'
 "I'm not sure if I'm going to be able to train my dog.\n"
 '\n'
 'A:\n'
 '\n'
 "I'm not sure if I'm going to be able to train my dog.\n"
 '\n'
 'A:\n'
 '\n'
 "I'm not sure if I'm going to be able to train my dog.\n"
 '\n'
 'A:\n'
 '\n'
 "I'm not sure if I'm going to be able to train my dog.\n"
 '\n'
 'A:\n'
 '\n'
 "I'm not sure if I'm going to be able to train my dog.\n"
 '\n'
 'A:\n'
 '\n'
 "I'm not sure if I'm going to be able to train my dog.\n"
 '\n'
 'A:\n'
 '\n'
 "I'm not sure if I'm going to be able t

Now lets see how the tokenizing works in general

In [27]:
list_texts = ["Hi, how are you?", "I'm good", "Yes"]
encoded_texts = tokenizer(list_texts)
print("Encoded several texts: ", encoded_texts["input_ids"])

Encoded several texts:  [[12764, 13, 849, 403, 368, 32], [42, 1353, 1175], [4374]]


Now lets make a dataset of our choice
We will be using the lamini docs by lamini dataset

In [28]:
finetuning_dataset_path = "lamini/lamini_docs"
finetuning_dataset = load_dataset(finetuning_dataset_path)
print(finetuning_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
})


In [29]:
dataset_path = "lamini/lamini_docs"
use_hf = True

In [30]:
model_name = "EleutherAI/pythia-70m"

In [31]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}

In [32]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = finetuning_dataset['train'],finetuning_dataset['test']

print(train_dataset)
print(test_dataset)

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1260
})
Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 140
})


In [33]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)

In [34]:
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

In [35]:
base_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
        

In [36]:
#Base model performance
#You can see the performance is very poor here 
test_text = test_dataset[0]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {test_dataset[0]['answer']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Can Lamini generate technical documentation or user manuals for software projects?
Correct answer from Lamini docs: Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.
Model's answer: 


d:\Dhruv_Sandhu\Machine Learning\Projects\Fine_tuning\finenv\lib\site-packages\transformers\models\gpt_neox\modeling_gpt_neox.py:460: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(




I have a question about the following:

How do I get the correct documentation to work?

A:

I think you need to use the following code:

A:

You can use the following code to get the correct documentation.

A:

You can use the following code to get the correct documentation.

A:

You can use the following


Sample Training

In [37]:
max_steps=280

In [38]:
trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

In [39]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=10,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Enables the progress bars shown during training
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

d:\Dhruv_Sandhu\Machine Learning\Projects\Fine_tuning\finenv\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [40]:
#It tells you the memory footprint and flops of the model 
model_flops = (
  base_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

print(base_model)
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
        

In [41]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

max_steps is given, it will override any value given in num_train_epochs


In [42]:
training_output = trainer.train()

  0%|          | 1/280 [00:00<02:55,  1.59it/s]

{'loss': 4.1563, 'grad_norm': 79.16706848144531, 'learning_rate': 1e-05, 'epoch': 0.0}


  1%|          | 2/280 [00:00<02:07,  2.18it/s]

{'loss': 3.0687, 'grad_norm': 60.92185592651367, 'learning_rate': 9.96415770609319e-06, 'epoch': 0.01}


  1%|          | 3/280 [00:01<01:47,  2.57it/s]

...

{'loss': 2.131, 'grad_norm': 51.78246307373047, 'learning_rate': 3.5842293906810036e-08, 'epoch': 0.89}


100%|██████████| 280/280 [01:41<00:00,  3.03it/s]

{'loss': 2.0542, 'grad_norm': 43.42436599731445, 'learning_rate': 0.0, 'epoch': 0.89}


100%|██████████| 280/280 [01:43<00:00,  2.71it/s]

{'train_runtime': 103.1553, 'train_samples_per_second': 10.857, 'train_steps_per_second': 2.714, 'train_loss': 2.4914458308901106, 'epoch': 0.89}


In [43]:
save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

Saved model to: lamini_docs_280_steps/final


In [44]:
print(device)

cuda


In [45]:
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)

In [46]:
finetuned_slightly_model.to(device) 

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
        

In [47]:
test_question = test_dataset[0]['question']
print("Question input (test):", test_question)

print("Finetuned slightly model's answer: ")
pprint(inference(test_question, finetuned_slightly_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Can Lamini generate technical documentation or user manuals for software projects?
Finetuned slightly model's answer: 
('Yes, Lamini can generate technical documentation or user manuals for '
 'software projects. It can generate technical documentation or user manuals '
 'for software projects. It can generate technical documentation or user '
 'manuals for software projects. It can generate technical documentation or '
 'user manuals for software projects. It can generate technical documentation '
 'or user manuals for software projects. It can generate technical '
 'documentation or user manuals for software projects. It can generate')


In [48]:
test_dataset

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 140
})

In [49]:
test_answer = test_dataset[0]['answer']
print("The target answer output the we were aiming for (test):", test_answer)

The target answer output the we were aiming for (test): Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.


In [50]:
print(base_model)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
        